## APLICACIÓN DE MODELOS DE REGLOG Y KMEANS PARA PREDICCIÓN DE RIESGO Y CLUSTERING

#### NOTEBOOK: Aplicar_Modelos

![Secuencia de Modelos ML](../Imagenes/01_Secuencia%20de%20Modelos%20ML.png)

Flujo de ejecución automatizada de modelos de machine learning en MS Fabric

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, when, count, isnan, isnull, avg, udf
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, DoubleType
import requests
import os
from datetime import datetime
import time

# Definir UDF para extraer probabilidad
@udf(returnType=DoubleType())
def get_prob_udf(probability_vector):
    """
    Extrae la probabilidad de aprobación (clase 1.0) del vector de probabilidades
    Args:
        probability_vector: Vector de probabilidades [p(0), p(1)]
    Returns:
        float: Probabilidad de aprobación (clase 1.0)
    """
    try:
        # El vector de probabilidad tiene el formato [p(0), p(1)]
        # Retornamos p(1) que es la probabilidad de aprobación
        return float(probability_vector[1])
    except:
        return None

# Verificar y obtener ruta del lakehouse
lakehouse_paths = [
    "Tables",  # Ruta relativa
    "/lakehouse/default/Tables",  # Ruta absoluta
    "abfss://lakehouse@onelake.dfs.fabric.microsoft.com/Tables"  # URL ABFS
]

def test_lakehouse_access(path):
    try:
        test_path = f"{path}/test_aplicacion"
        test_df = spark.createDataFrame([(1,)], ["test"])
        test_df.write.mode("overwrite").format("delta").save(test_path)
        print(f"Escritura exitosa en {test_path}")
        return True, test_path
    except Exception as e:
        print(f"Error al escribir en {test_path}: {str(e)}")
        return False, None

lakehouse_path = None
for path in lakehouse_paths:
    success, test_path = test_lakehouse_access(path)
    if success:
        lakehouse_path = path
        break

if not lakehouse_path:
    raise Exception("No se pudo acceder a ninguna ruta del lakehouse. Verifica los permisos y la configuración.")

print(f"Usando ruta del lakehouse: {lakehouse_path}")

# Función para registrar logs
def log_reentrenamiento(step_name, status, metrics=None, error_message=None):
    log_schema = StructType([
        StructField("fecha_ejecucion", TimestampType(), False),
        StructField("paso", StringType(), False),
        StructField("estado", StringType(), False),
        StructField("error_mensaje", StringType(), True),
        StructField("metricas", StringType(), True)
    ])
    log_df = spark.createDataFrame([(
        datetime.now(),
        step_name,
        status,
        error_message,
        str(metrics) if metrics else None
    )], schema=log_schema)
    try:
        log_df.write.mode("append").format("delta").save(f"{lakehouse_path}/reentrenamiento_log")
        print(f"Log registrado para {step_name}: {status}")
    except Exception as e:
        print(f"Error al registrar log: {e}")

# Función para notificaciones Discord
discord_webhook_url = os.getenv('DISCORD_WEBHOOK_URL', "XXXXXXXXXXXXXXXXXXXX")

def crear_mensaje(step_name, status, metrics=None, error_message=None):
    color = 3066993 if status == "Éxito" else 15158332
    description = f"Aplicación de {step_name} completada con estado: {status}"
    if error_message:
        description += f"\nError: {error_message}"
    
    embed = {
        "title": f"{'✅' if status == 'Éxito' else '❌'} Aplicación de Modelos - {step_name}",
        "description": description,
        "color": color,
        "footer": {"text": "RiskApp - Sistema de Monitoreo"},
        "timestamp": datetime.now().isoformat()
    }
    
    if metrics:
        embed["fields"] = [
            {"name": key, "value": f"{value:.4f}", "inline": True}
            for key, value in metrics.items()
        ]
    
    return {"embeds": [embed]}

def enviar_notificacion(mensaje):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(
                discord_webhook_url, 
                json=mensaje,
                timeout=10  # Agregar timeout
            )
            response.raise_for_status()
            print(f"Notificación enviada para {mensaje['embeds'][0]['title']}")
            return
        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:  # Si es el último intento
                print(f"Error al enviar notificación después de {max_retries} intentos: {str(e)}")
            else:
                print(f"Intento {attempt + 1} fallido, reintentando...")
                time.sleep(1)  # Esperar 1 segundo antes de reintentar

# Función para verificar existencia y permisos de tabla Delta
def check_delta_table(path, table_name):
    try:
        # Intentar leer la tabla
        spark.read.format("delta").load(f"{path}/{table_name}")
        return True, None
    except Exception as e:
        error_msg = f"Error accediendo a la tabla {table_name}: {str(e)}"
        return False, error_msg

# Función para validar el esquema y tipos de datos
def validate_table_schema(df, table_name):
    try:
        # Validar tipos de datos para compatibilidad SQL
        for column, dtype in df.dtypes:
            if 'vector' in str(dtype).lower() or 'array' in str(dtype).lower():
                raise Exception(f"Columna {column} tiene tipo {dtype} no compatible con SQL")
        return True, None
    except Exception as e:
        error_msg = f"Error de validación de esquema para {table_name}: {str(e)}"
        return False, error_msg

# Función mejorada para escribir en tabla Delta
def write_delta_table_with_validation(df, path, table_name, mode="overwrite", max_retries=3):
    # Validar esquema primero
    schema_valid, schema_error = validate_table_schema(df, table_name)
    if not schema_valid:
        return False, schema_error
    
    # Asegurar que los tipos de datos sean compatibles con SQL
    try:
        # Convertir columnas específicas a tipos compatibles
        if table_name == "predicciones_pendientes":
            df = df.select(
                "id_cliente",
                "estado_solicitud",
                col("prediction").cast("double"),
                col("prediccion_aprobado").cast("string"),
                col("score_aprobacion").cast("double")
            )
    except Exception as e:
        return False, f"Error al convertir tipos de datos: {str(e)}"
    
    # Intentar escribir con reintentos
    for attempt in range(max_retries):
        try:
            df.write.mode(mode) \
                .format("delta") \
                .option("overwriteSchema", "true") \
                .option("mergeSchema", "true") \
                .save(f"{path}/{table_name}")
            return True, None
        except Exception as e:
            if attempt == max_retries - 1:
                error_msg = f"Error escribiendo en tabla {table_name} después de {max_retries} intentos: {str(e)}"
                return False, error_msg
            time.sleep(2 ** attempt)

# Función mejorada para notificaciones Discord
def enviar_notificacion_tabla(table_name, status, error_message=None):
    color = 3066993 if status == "Éxito" else 15158332
    description = f"Operación en tabla {table_name}: {status}"
    if error_message:
        description += f"\nError: {error_message}"
    
    embed = {
        "title": f"{'✅' if status == 'Éxito' else '❌'} Estado Tabla - {table_name}",
        "description": description,
        "color": color,
        "footer": {"text": "RiskApp - Sistema de Monitoreo de Tablas"},
        "timestamp": datetime.now().isoformat()
    }
    
    try:
        response = requests.post(
            discord_webhook_url,
            json={"embeds": [embed]},
            timeout=10
        )
        response.raise_for_status()
        print(f"Notificación enviada para tabla {table_name}")
    except Exception as e:
        print(f"Error al enviar notificación para tabla {table_name}: {str(e)}")

# Verificar todas las tablas necesarias al inicio
required_tables = ["solicitudes_processed", "predicciones_pendientes", "solicitudes_clustered"]
table_status = {}

for table in required_tables:
    success, error = check_delta_table(lakehouse_path, table)
    table_status[table] = {"exists": success, "error": error}
    if not success:
        enviar_notificacion_tabla(table, "Error", error)

# Cargar y preparar datos
try:
    df = spark.read.format("delta").load(f"{lakehouse_path}/solicitudes_processed")
    total_records = df.count()
    
    # Verificar valores nulos por tipo de dato
    null_counts = {}
    for col_name, dtype in df.dtypes:
        if dtype in ("double", "float"):
            null_count = df.filter(col(col_name).isNull() | isnan(col(col_name))).count()
        else:
            null_count = df.filter(col(col_name).isNull()).count()
        null_counts[col_name] = null_count
    
    print(f"Datos cargados. Total de registros: {total_records}")
    print("\nAnálisis de valores nulos:")
    for col_name, null_count in null_counts.items():
        null_percentage = (null_count / total_records) * 100
        print(f"{col_name}: {null_count} nulos ({null_percentage:.2f}%)")
    
    # Imputar valores nulos con la mediana
    numeric_cols = ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual", "antiguedad_laboral"]
    for col_name in numeric_cols:
        median_value = df.approxQuantile(col_name, [0.5], 0.01)[0]
        df = df.fillna(median_value, subset=[col_name])
    
    # Rellenar valores nulos en columnas codificadas
    encoded_cols = ["historial_pagos_encoded", "estado_civil_encoded", "tipo_empleo_encoded"]
    df = df.fillna(0, subset=encoded_cols)
    
    log_reentrenamiento("Carga_datos", "Éxito", metrics={"total_records": total_records})
    mensaje = crear_mensaje("Carga_datos", "Éxito", metrics={"total_records": float(total_records)})
    enviar_notificacion(mensaje)
except Exception as e:
    error_msg = str(e)
    log_reentrenamiento("Carga_datos", "Error", error_message=error_msg)
    mensaje = crear_mensaje("Carga_datos", "Error", error_message=error_msg)
    enviar_notificacion(mensaje)
    raise Exception(f"Error al cargar datos: {error_msg}")

# Aplicar clustering
try:
    print("Cargando modelo K-means...")
    kmeans_model = KMeansModel.load(f"{lakehouse_path}/Models/KMeansClustering")
    
    # Preparar características para clustering
    feature_cols = ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual",
                   "antiguedad_laboral", "historial_pagos_encoded", "estado_civil_encoded",
                   "tipo_empleo_encoded", "numero_dependientes"]
    
    # Configurar preprocesamiento
    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features",
        handleInvalid="skip"
    )
    
    scaler = StandardScaler(
        inputCol="features",
        outputCol="scaled_features",
        withStd=True,
        withMean=True
    )
    
    print("Aplicando transformación de clustering...")
    # Aplicar preprocesamiento y clustering
    assembled_df = assembler.transform(df)
    scaled_df = scaler.fit(assembled_df).transform(assembled_df)
    clustered_df = kmeans_model.transform(scaled_df)
    
    # Analizar resultados
    cluster_distribution = clustered_df.groupBy("prediction").count().toPandas()
    print("\nDistribución de clusters:")
    print(cluster_distribution)
    
    # Guardar resultados - Corregido para usar id_cliente
    result_df = clustered_df.select(
        "id_cliente",  # Cambiado de id_solicitud a id_cliente
        "estado_solicitud",
        col("prediction").alias("cluster")
    )
    
    success, error = write_delta_table_with_validation(result_df, lakehouse_path, "solicitudes_clustered")
    if not success:
        raise Exception(error)
    
    verify_success, verify_error = check_delta_table(lakehouse_path, "solicitudes_clustered")
    if not verify_success:
        raise Exception(f"Error verificando tabla después de escritura: {verify_error}")
    
    metrics = {
        "total_clustered": float(result_df.count()),
        "num_clusters": float(cluster_distribution.shape[0])
    }
    
    try:
        log_reentrenamiento("Clustering", "Éxito", metrics=metrics)
        mensaje = crear_mensaje("Clustering", "Éxito", metrics=metrics)
        enviar_notificacion(mensaje)
        enviar_notificacion_tabla("solicitudes_clustered", "Éxito")
    except Exception as notif_error:
        print(f"Error al enviar notificación: {str(notif_error)}")
        
except Exception as e:
    error_msg = str(e)
    try:
        log_reentrenamiento("Clustering", "Error", error_message=error_msg)
        mensaje = crear_mensaje("Clustering", "Error", error_message=error_msg)
        enviar_notificacion(mensaje)
        enviar_notificacion_tabla("solicitudes_clustered", "Error", error_msg)
    except Exception as notif_error:
        print(f"Error al enviar notificación de error: {str(notif_error)}")
    raise Exception(f"Error al aplicar clustering: {error_msg}")

# Antes de la sección de clasificación, agregar función de validación SQL
def ensure_sql_compatibility(df, table_name):
    """
    Asegura que los tipos de datos sean compatibles con SQL Server
    """
    sql_compatible_schema = {
        "predicciones_pendientes": {
            "id_cliente": "string",
            "estado_solicitud": "string",
            "prediction": "double",
            "prediccion_aprobado": "string",
            "score_aprobacion": "double"
        }
    }
    
    if table_name in sql_compatible_schema:
        required_schema = sql_compatible_schema[table_name]
        for col_name, required_type in required_schema.items():
            if col_name not in df.columns:
                raise ValueError(f"Columna requerida {col_name} no encontrada en el DataFrame")
            df = df.withColumn(col_name, col(col_name).cast(required_type))
    return df

# Aplicar clasificación
try:
    print("Cargando modelo de clasificación...")
    clf_model = PipelineModel.load(f"{lakehouse_path}/Models/ClasificadorCredito")
    print("Filtrando solicitudes pendientes...")
    pending_df = df.filter(col("solicitud_credito").isNull())
    pending_count = pending_df.count()
    
    if pending_count > 0:
        print(f"Procesando {pending_count} solicitudes pendientes...")
        predictions = clf_model.transform(pending_df)
        
        # Analizar predicciones
        pred_distribution = predictions.groupBy("prediction").count().toPandas()
        print("\nDistribución de predicciones:")
        print(pred_distribution)
        
        # Preparar resultados finales con tipos de datos estrictamente compatibles con SQL
        final_predictions = predictions.select(
            col("id_cliente").cast("string"),
            col("estado_solicitud").cast("string"),
            col("prediction").cast("double"),
            when(col("prediction") == 1.0, "Aprobado")
            .otherwise("Rechazado").alias("prediccion_aprobado"),
            get_prob_udf("probability").alias("score_aprobacion").cast("double")
        )
        
        # Validar y asegurar compatibilidad SQL
        final_predictions = ensure_sql_compatibility(final_predictions, "predicciones_pendientes")
        
        # Verificar que no haya valores nulos
        null_check = final_predictions.select([
            count(when(col(c).isNull() | isnan(col(c)), c)).alias(c)
            for c in final_predictions.columns
        ]).first()
        
        null_columns = [col for col, count in null_check.asDict().items() if count > 0]
        if null_columns:
            raise ValueError(f"Valores nulos encontrados en columnas: {', '.join(null_columns)}")
        
        # Verificar rangos de valores
        validation_checks = final_predictions.select(
            (count(when((col("prediction") != 0.0) & (col("prediction") != 1.0), 1)) == 0).alias("valid_prediction"),
            (count(when((col("score_aprobacion") < 0.0) | (col("score_aprobacion") > 1.0), 1)) == 0).alias("valid_score")
        ).first()
        
        if not (validation_checks["valid_prediction"] and validation_checks["valid_score"]):
            raise ValueError("Valores fuera de rango detectados en prediction o score_aprobacion")
        
        # Intentar escribir con validación mejorada
        success, error = write_delta_table_with_validation(final_predictions, lakehouse_path, "predicciones_pendientes")
        if not success:
            error_detail = f"DeltaTableUserException: {error}"
            enviar_notificacion_tabla("predicciones_pendientes", "Error", error_detail)
            raise Exception(error_detail)
        
        # Verificar la tabla después de escribir
        verify_success, verify_error = check_delta_table(lakehouse_path, "predicciones_pendientes")
        if not verify_success:
            error_detail = f"Error de verificación post-escritura: {verify_error}"
            enviar_notificacion_tabla("predicciones_pendientes", "Error", error_detail)
            raise Exception(error_detail)
        
        avg_score = final_predictions.agg(avg("score_aprobacion").alias("avg_score")).collect()[0]["avg_score"]
        
        metrics = {
            "total_pendientes": float(pending_count),
            "aprobados_predichos": float(pred_distribution[pred_distribution["prediction"] == 1.0]["count"].iloc[0]),
            "score_promedio": float(avg_score)
        }
        
        log_reentrenamiento("Clasificación", "Éxito", metrics=metrics)
        mensaje = crear_mensaje("Clasificación", "Éxito", metrics=metrics)
        enviar_notificacion_tabla("predicciones_pendientes", "Éxito")
    else:
        mensaje = crear_mensaje("Clasificación", "Éxito", error_message="No hay solicitudes pendientes")
        log_reentrenamiento("Clasificación", "Éxito", error_message="No hay solicitudes pendientes")
    
    enviar_notificacion(mensaje)
except Exception as e:
    error_msg = str(e)
    log_reentrenamiento("Clasificación", "Error", error_message=error_msg)
    mensaje = crear_mensaje("Clasificación", "Error", error_message=error_msg)
    enviar_notificacion(mensaje)
    enviar_notificacion_tabla("predicciones_pendientes", "Error", error_msg)
    print(f"Error detallado: {error_msg}")
    raise Exception(f"Error al aplicar clasificación o escribir tabla: {error_msg}")

# Notificación final
try:
    mensaje = crear_mensaje(
        "Proceso Completo",
        "Éxito",
        metrics={
            "total_registros": float(total_records),
            "solicitudes_pendientes": float(pending_count if 'pending_count' in locals() else 0)
        }
    )
    enviar_notificacion(mensaje)
except Exception as e:
    print(f"Error al enviar notificación final: {str(e)}")

## Respuesta de aplicación automatizada de modelos en MS Fabric

![Respuesta de Aplicacion de Modelos](../Imagenes/00_Respuesta%20de%20Aplicacion%20de%20Modelos.png)